## notMNIST資料集分類

notMNIST dataset consist of 10 classes. Each image is a letter(from A-J). Following are examples of letter "A"

![Alt text](./images/cnn_implement/notmnist.png)

more reference http://yaroslavvb.blogspot.tw/2011/09/notmnist-dataset.html


We first download dataset. Following is download script.

In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

'''
Reformat into a TensorFlow-friendly shape:
    convolutions need the image data formatted as a cube (width by height by #channels)
    labels as float 1-hot encodings.
'''
def reformat(dataset, labels):      #將dataset轉成 4D張量, 
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels 

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('After reformat......')
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters                                                                      #depth = 16 here
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) 
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    
    # Flatten previous layer result
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])

    # Feed into fully connected layer
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

num_steps = 2001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = { x: batch_data, y: batch_labels}
        _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step {}'.format(step, l))
            print('Minibatch accuracy: {}'.format(train_accuracy_))
    
    '''
    Testing Part
    '''        
    print('Testing......')
    feed_dict = { x: test_dataset, y: test_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))


# Cifar10資料集分類

In [1]:
import tensorflow as tf
import numpy as np
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='latin1')
    return d
    

def onehot(labels):
    '''Get one-hot encoding '''
    n_sample = len(labels)
    n_class = max(labels) + 1
    onehot_labels = np.zeros((n_sample, n_class))
    onehot_labels[np.arange(n_sample), labels] = 1

    return onehot_labels

def convert_images(raw):
    '''
    Convert images from the CIFAR-10 format and
    return a 4-dim array with shape: [image_number, height, width, channel]
    where the pixels are floats between 0.0 and 1.0.
    '''

    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw, dtype=float) / 255.0

    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, 3, 32, 32])

    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])

    return images


data1 = unpickle('cifar-10-batches-py/data_batch_1')
data2 = unpickle('cifar-10-batches-py/data_batch_2')
data3 = unpickle('cifar-10-batches-py/data_batch_3')
data4 = unpickle('cifar-10-batches-py/data_batch_4')
data5 = unpickle('cifar-10-batches-py/data_batch_5')

# preprocess all training data and labels
x_train = np.concatenate((data1['data'], data2['data'], data3['data'], data4['data'], data5['data']), axis=0)
x_train = convert_images(x_train)
label = np.concatenate((data1['labels'], data2['labels'], data3['labels'], data4['labels'], data5['labels']), axis=0)
y_train = onehot(label)

# preprocess all testing data and labels
test = unpickle('cifar-10-batches-py/test_batch')
x_test = test['data'] 
x_test = convert_images(x_test)
y_test = onehot(test['labels'])


print(x_train.shape)
print(x_test.shape)

total_epoch = 40
image_size =32
batch_size = 32
beta = 0.01
num_channels = 3
num_labels = 10
learning_rate = 0.001

x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

'''Build model'''

layer1_weights = tf.Variable(tf.truncated_normal([3, 3, num_channels, 32], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([32]))
conv1 = tf.nn.conv2d(x, layer1_weights, [1, 2, 2, 1], padding='SAME')
hidden1 = tf.nn.relu(conv1 + layer1_biases)
pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


layer2_weights = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[64]))
conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 2, 2, 1], padding='SAME')
hidden2 = tf.nn.relu(conv2 + layer2_biases)
pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#flatten
shape = pool2.get_shape().as_list()
reshape = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])

# FC part
layer3_weights = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], 64], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[64]))
hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)



layer4_weights = tf.Variable(tf.truncated_normal([64, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
logits =  tf.matmul(hidden3, layer4_weights) + layer4_biases

'''Define loss'''  #損失函數
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)\
                      +beta*tf.nn.l2_loss(layer1_weights)
                      +beta*tf.nn.l2_loss(layer2_weights)
                      +beta*tf.nn.l2_loss(layer3_weights)
                      +beta*tf.nn.l2_loss(layer4_weights)
                     
                     )

'''Optimization'''
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('start training......')
    for epoch in range(total_epoch):
        # calculate total batch within a epoch
        num_steps =  y_train.shape[0] // batch_size
        avg_cost = 0 
        avg_acc = 0.0
        for step in range(num_steps):
            offset = (step * batch_size) % (y_train.shape[0] - batch_size)
            batch_data = x_train[offset:(offset + batch_size), :, :, :]
            batch_labels = y_train[offset:(offset + batch_size), :]
            feed_dict = { x: batch_data, y: batch_labels}
            _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
            avg_cost = avg_cost + (l/num_steps)
            avg_acc = avg_acc + (train_accuracy_/num_steps)
        
        if epoch % 20 == 0:
            print('loss at epoch {}:\ncost: {}, accuracy: {}'.format(epoch, avg_cost, avg_acc))
    
    '''
    Testing Part
    '''        
    print('start testing......')
    feed_dict = { x: x_test, y: y_test}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))

(50000, 32, 32, 3)
(10000, 32, 32, 3)
start training......
loss at epoch 0:
cost: 2.304502329737828, accuracy: 0.3444102112676068
loss at epoch 20:
cost: 1.5922623901757915, accuracy: 0.5483354673495539
start testing......
Test accuracy: 0.5460000038146973
